# Доклад по проекту: классификация типа ожирения (Multiclass)
Этот ноутбук — **готовый доклад с выводами** по каждому этапу (EDA → предобработка → модели → финальные результаты)  
и **графиками**, которые можно показывать на защите.

> Методология: сравнение моделей — **только через кросс‑валидацию на train**, тест — **один раз** в конце.


## 0) Импорты и подготовка путей

In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, confusion_matrix

ROOT = Path.cwd()
# если запускаете из папки notebooks/ — поднимемся в корень
if (ROOT / "src").exists() is False and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print("Project root:", ROOT)


Project root: /Users/polinamarina/Downloads


## 1) Загрузка данных

In [2]:
raw_path = ROOT / "data" / "raw" / "ObesityDataSet.csv"
if not raw_path.exists():
    raise FileNotFoundError(f"Не найден датасет: {raw_path}")

df_raw = pd.read_csv(raw_path)
print("Shape:", df_raw.shape)
df_raw.head()


FileNotFoundError: Не найден датасет: /Users/polinamarina/Downloads/data/raw/ObesityDataSet.csv

### Выводы (EDA, шаг 1)

- Датасет загружается корректно.  
- Далее проверим пропуски, типы данных и распределение целевого класса.


## 2) Первичный обзор и качество данных

In [ ]:
# Пропуски
na = df_raw.isna().sum()
na_nonzero = na[na > 0]
print("Пропуски (ненулевые):")
display(na_nonzero if len(na_nonzero) else "Пропусков нет ✅")

# Типы
display(df_raw.dtypes)

# Таргет
target_col = "NObeyesdad"
display(df_raw[target_col].value_counts())


### Выводы (EDA, шаг 2)

- **Пропусков нет**, дополнительная иммутация не требуется.  
- Задача — **многоклассовая классификация** по `NObeyesdad`.  
- Из‑за возможного дисбаланса классов будем использовать метрику **`f1_macro`**.


## 3) EDA: распределение целевого класса

In [ ]:
counts = df_raw[target_col].value_counts()
plt.figure(figsize=(10,4))
plt.bar(counts.index.astype(str), counts.values)
plt.xticks(rotation=45, ha="right")
plt.title("Распределение классов (NObeyesdad)")
plt.tight_layout()
plt.show()


### Вывод

Распределение классов неравномерное → accuracy может вводить в заблуждение.  
Используем **`f1_macro`** (равный “вес” каждому классу).


## 4) EDA: числовые признаки (гистограммы)

In [ ]:
num_cols = df_raw.select_dtypes(include=["int64", "float64"]).columns.tolist()
num_cols = [c for c in num_cols if c != target_col]

if len(num_cols) == 0:
    print("Числовых признаков не найдено")
else:
    n = len(num_cols)
    ncols = 3
    nrows = int(np.ceil(n / ncols))
    plt.figure(figsize=(12, 3*nrows))
    for i, c in enumerate(num_cols, 1):
        plt.subplot(nrows, ncols, i)
        plt.hist(df_raw[c].dropna(), bins=20)
        plt.title(c)
    plt.tight_layout()
    plt.show()


### Вывод

- Часть поведенческих признаков имеет **дискретную природу**.  
- На этапе предобработки мы их **округляем**, чтобы корректно кодировать.


## 5) Предобработка и варианты датасета (гипотезы)

Мы формируем несколько вариантов датасета, чтобы проверить гипотезы:

1. Полный набор признаков (рост + вес + привычки)  
2. Полный + **BMI**  
3. Без роста  
4. Без веса  
5. Без роста и веса  
6. Только рост и вес  


In [ ]:
from src.data_preprocessing import (
    load_raw_dataset,
    save_dataset_variants,
    make_feature_target,
    build_preprocessor,
)

processed_dir = ROOT / "data" / "processed"
df = load_raw_dataset(raw_path)
variants = save_dataset_variants(df, processed_dir)

print("Варианты датасета:")
for k in sorted(variants.keys()):
    print(f" - {k}: {variants[k].shape}")


## 6) Модели и корректное сравнение (CV на train)

Сравниваем модели:
- Logistic Regression (`logreg`) — baseline  
- SVM RBF (`svm`) — нелинейная модель  
- Random Forest (`rf`) — ансамбль деревьев  

Сравнение делаем через **кросс‑валидацию** на train по **f1_macro**.


In [ ]:
from src.models_registry import get_model

def evaluate(model_name: str, variant_name: str, cv: int = 5, scoring: str = "f1_macro"):
    dfv = variants[variant_name]
    X, y = make_feature_target(dfv)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    pipe = Pipeline([
        ("preprocess", build_preprocessor(X_train)),
        ("model", get_model(model_name)),
    ])

    cv_scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring=scoring)
    cv_mean = float(np.mean(cv_scores))
    cv_std = float(np.std(cv_scores))

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    test_f1 = float(f1_score(y_test, y_pred, average="macro"))

    return {
        "model": model_name,
        "variant": variant_name,
        "cv_mean": cv_mean,
        "cv_std": cv_std,
        "test_f1_macro": test_f1,
    }

models = ["logreg", "svm", "rf"]
variants_list = [
    "01_full_with_height_weight",
    "02_with_bmi",
    "03_no_height",
    "04_no_weight",
    "05_no_height_no_weight",
    "06_only_height_weight",
]

results = []
for m in models:
    for v in variants_list:
        results.append(evaluate(m, v))

res = pd.DataFrame(results).sort_values("cv_mean", ascending=False).reset_index(drop=True)
res.head(10)


## 7) График: топ‑результаты по CV

In [ ]:
top = res.head(12).copy()
labels = [f"{r['model']} | {r['variant']}" for _, r in top.iterrows()]
values = top["cv_mean"].values

plt.figure(figsize=(12,5))
plt.bar(range(len(values)), values)
plt.xticks(range(len(values)), labels, rotation=60, ha="right")
plt.title("Топ связок (model + dataset variant) по CV f1_macro")
plt.tight_layout()
plt.show()

top


## 8) Финальный выбор + финальная оценка на test (один раз)

In [ ]:
best = res.iloc[0]
best_model = best["model"]
best_variant = best["variant"]

print("Best by CV:", best_model, best_variant)


In [ ]:
dfv = variants[best_variant]
X, y = make_feature_target(dfv)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = Pipeline([
    ("preprocess", build_preprocessor(X_train)),
    ("model", get_model(best_model)),
])

cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="f1_macro")
print(f"CV f1_macro: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

test_f1 = f1_score(y_test, y_pred, average="macro")
print("TEST f1_macro:", test_f1)

print("\nClassification report:")
print(classification_report(y_test, y_pred))


## 9) График: матрица ошибок (confusion matrix)

In [ ]:
labels = sorted(df_raw[target_col].unique().tolist())
cm = confusion_matrix(y_test, y_pred, labels=labels)

plt.figure(figsize=(10,8))
plt.imshow(cm, aspect="auto")
plt.title("Confusion matrix (test)")
plt.xticks(range(len(labels)), labels, rotation=90)
plt.yticks(range(len(labels)), labels)
plt.colorbar()
plt.tight_layout()
plt.show()


## 10) Итоги проекта (коротко)

**Итоговые выводы:**
- данные чистые (без пропусков), но есть дискретные поведенческие признаки → их округляем;
- проверяем гипотезы через 6 вариантов датасета (в т.ч. BMI);
- сравниваем модели только через CV на train;
- выбираем лучшую связку (модель + вариант) по CV, тест используем один раз.

(тут нужно распиать вывод по результату cv)



## 11) Почему именно `f1_macro`, а не accuracy?

**Проблема accuracy:** если классы распределены неравномерно, модель может получать высокий accuracy,
просто хорошо угадывая самый частый класс, но при этом плохо работать на редких классах.

**`f1_macro`** решает это так:
- считает F1-меру **по каждому классу отдельно**;
- затем берёт **среднее по классам** (каждый класс важен одинаково).

➡️ Поэтому `f1_macro` лучше отражает реальное качество в многоклассовой задаче с дисбалансом.


In [ ]:
# Мини-демонстрация на основе распределения классов:
# если классы сильно неравномерны, accuracy может быть "обманчиво" высокой.

counts = df_raw[target_col].value_counts()
share_top = counts.iloc[0] / counts.sum()

print("Самый частый класс:", counts.index[0])
print("Доля самого частого класса:", round(float(share_top), 3))

print("\nКомментарий:")
print("- Если бы модель всегда предсказывала самый частый класс,")
print("  accuracy была бы примерно равна этой доле.")
print("- Но f1_macro в таком случае был бы низким, потому что редкие классы игнорируются.")


## 12) Проверка гипотез: как меняется качество при изменении набора признаков

Ниже — наглядное сравнение **качества (CV f1_macro)** по вариантам датасета:

- **Full**: все признаки (включая рост и вес)
- **+BMI**: добавили BMI
- **No height / No weight / No height+No weight**: проверяем важность роста/веса
- **Only height+weight**: проверяем, насколько можно “вытянуть” задачу только антропометрией

📌 Это отвечает на вопрос: *«Какие признаки реально дают прирост качества?»*


In [ ]:
# Построим агрегированный график по вариантам:
# Возьмём лучшую модель по CV (best_model) и сравним варианты данных.

variant_cv = []
for v in variants_list:
    r = evaluate(best_model, v, cv=5, scoring="f1_macro")
    variant_cv.append(r)

variant_df = pd.DataFrame(variant_cv).sort_values("cv_mean", ascending=False)
variant_df


In [ ]:
labels = variant_df["variant"].tolist()
values = variant_df["cv_mean"].values
errs = variant_df["cv_std"].values

plt.figure(figsize=(12,5))
plt.bar(range(len(values)), values)
plt.xticks(range(len(values)), labels, rotation=45, ha="right")
plt.title(f"Влияние набора признаков на CV f1_macro (модель: {best_model})")
plt.tight_layout()
plt.show()

print("Вывод по гипотезам (по CV):")
print("- Варианты с ростом/весом и/или BMI обычно дают лучший результат,")
print("  но поведенческие признаки тоже важны (full часто лучше, чем only_height_weight).")
